In [1]:
import numpy as np
from tensorflow import keras
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt
import pickle
import heapq

In [2]:
path = 'sher.txt'
text = open(path).read().lower()
# text = text.replace('\n', '').replace('\r', '').replace('\ufeff', '')
text = text.replace('\r', '').replace('\ufeff', '')
print('corpus length:', len(text))

corpus length: 571393


In [3]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [4]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [5]:
WORD_LENGTH = 3
prev_words = []
next_words = []
for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0])
print(next_words[0])

['i', 'need', 'help']
i


In [6]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

In [8]:
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=30, shuffle=True).history

Epoch 1/30
801/801 [==============================] - 80s 98ms/step - loss: 6.0273 - accuracy: 0.1068 - val_loss: 5.7110 - val_accuracy: 0.1278
Epoch 2/30
801/801 [==============================] - 79s 98ms/step - loss: 5.7582 - accuracy: 0.1442 - val_loss: 5.9890 - val_accuracy: 0.1308
Epoch 3/30
801/801 [==============================] - 79s 98ms/step - loss: 5.7535 - accuracy: 0.1727 - val_loss: 6.1189 - val_accuracy: 0.1347
Epoch 4/30
801/801 [==============================] - 79s 98ms/step - loss: 5.4843 - accuracy: 0.2045 - val_loss: 6.2120 - val_accuracy: 0.1306
Epoch 5/30
801/801 [==============================] - 80s 99ms/step - loss: 5.2029 - accuracy: 0.2387 - val_loss: 6.2211 - val_accuracy: 0.1308
Epoch 6/30
801/801 [==============================] - 79s 99ms/step - loss: 4.9931 - accuracy: 0.2690 - val_loss: 6.3346 - val_accuracy: 0.1325
Epoch 7/30
801/801 [==============================] - 79s 98ms/step - loss: 4.8299 - accuracy: 0.2995 - val_loss: 6.3731 - val_accuracy:

In [14]:
model.save('keras_next_word_model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('keras_next_word_model.h5')
history = pickle.load(open("history.p", "rb"))

In [15]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word, unique_word_index[word])
        x[0, t, unique_word_index[word]] = 1
    return x
prepare_input("It is not".lower())

it 3905
is 3896
not 4773


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [16]:
def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [17]:
def predict_completions(text, n=3):
    if text == "":
        return("0")
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [18]:
q =  "I need your help"
print("correct sentence: ",q)
seq = " ".join(tokenizer.tokenize(q.lower())[0:3])
print("Sequence: ",seq)
print("next possible words: ", predict_completions(seq, 12))

correct sentence:  I need your help
Sequence:  i need your
i 3617
need 4703
your 8010
next possible words:  ['house', 'father', 'case', 'business', 'morning', 'point', 'face', 'address', 'thing', 'right', 'name', 'room']
